In [1]:
import pandas as pd

In [2]:
# Read Dataset
growth = pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-industry-employment-growth.xlsx",sheet_name='Growth from Industry Transition')
skill_pen = pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-skill-penetration.xlsx",sheet_name='Skill Penetration')
ind_skill = pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-industry-skills-needs.xlsx",sheet_name='Industry Skills Needs')

In [3]:
industry=pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-talent-migration.xlsx", sheet_name='Industry Migration')
skill = pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-talent-migration.xlsx", sheet_name='Skill Migration')
country = pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/public_use-talent-migration.xlsx", sheet_name='Country Migration')

In [4]:
education_major= pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/education_and_major.xlsm",sheet_name="Major")
education_skill= pd.read_excel("/Users/tanya/Desktop/IS590 OM/OM DATASETS/education_and_major.xlsm",sheet_name="Skill")

In [5]:
# Select Certain Columns to make the new dataset
growth_from_ind=growth[['country_name','wb_income','wb_region','isic_section_name','industry_name','growth_rate_2015','growth_rate_2016','growth_rate_2017','growth_rate_2018']]
industry_migration=industry[['country_name','wb_income','wb_region','isic_section_name','industry_name','net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018']]
skill_migration=skill[['country_name','wb_income','wb_region','skill_group_category','skill_group_name','net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018']]


In [6]:
skill_penetration=skill_pen[['year','isic_section_name','industry_name','skill_group_category','skill_group_name','skill_group_penetration_rate']]
ind_skill_needs=ind_skill[['year','isic_section_name','industry_name','skill_group_category','skill_group_name','skill_group_rank']]

In [7]:
# Check Each Column's Data Type of Each Dataset
growth_from_ind.dtypes
skill_penetration.dtypes
ind_skill_needs.dtypes
industry_migration.dtypes
skill_migration.dtypes

country_name             object
wb_income                object
wb_region                object
skill_group_category     object
skill_group_name         object
net_per_10K_2015        float64
net_per_10K_2016        float64
net_per_10K_2017        float64
net_per_10K_2018        float64
dtype: object

In [8]:
# Delete all the null value
skill_penetration=skill_penetration[pd.notnull(skill_penetration['skill_group_penetration_rate'])]
ind_skill_needs=ind_skill_needs[pd.notnull(ind_skill_needs['skill_group_rank'])]

In [9]:
growth_from_ind=growth_from_ind[pd.notnull(growth_from_ind['growth_rate_2015'])]
growth_from_ind=growth_from_ind[pd.notnull(growth_from_ind['growth_rate_2016'])]
growth_from_ind=growth_from_ind[pd.notnull(growth_from_ind['growth_rate_2017'])]
growth_from_ind=growth_from_ind[pd.notnull(growth_from_ind['growth_rate_2018'])]

In [10]:
# Combine Various Dataset

In [11]:
# Step 1 combine skill penetration and industry skill needs
skill_part=pd.merge(skill_penetration,ind_skill_needs, how='outer', on=['year','isic_section_name','industry_name','skill_group_category','skill_group_name'])

In [12]:
df1 = skill_part[skill_part['year'] == 2015].drop(columns=['year'])
df2 = skill_part[skill_part['year'] == 2016].drop(columns=['year'])
df3 = skill_part[skill_part['year'] == 2017].drop(columns=['year'])
df4 = skill_part[skill_part['year'] == 2018].drop(columns=['year'])

In [13]:
df1=df1.rename(columns={'skill_group_penetration_rate':'skill_group_penetration_rate_2015','skill_group_rank':'skill_group_rank_2015'})
df2=df2.rename(columns={'skill_group_penetration_rate':'skill_group_penetration_rate_2016','skill_group_rank':'skill_group_rank_2016'})
df3=df3.rename(columns={'skill_group_penetration_rate':'skill_group_penetration_rate_2017','skill_group_rank':'skill_group_rank_2017'})
df4=df4.rename(columns={'skill_group_penetration_rate':'skill_group_penetration_rate_2018','skill_group_rank':'skill_group_rank_2018'})

In [14]:
final_skill_part=pd.merge(df1,df2, how='outer',on=['isic_section_name','industry_name','skill_group_category','skill_group_name'])
final_skill_part=pd.merge(final_skill_part,df3, how='outer',on=['isic_section_name','industry_name','skill_group_category','skill_group_name'])
final_skill_part=pd.merge(final_skill_part,df4, how='outer',on=['isic_section_name','industry_name','skill_group_category','skill_group_name'])

In [15]:
final_skill_part=final_skill_part[['isic_section_name','industry_name','skill_group_category','skill_group_name',"skill_group_penetration_rate_2015",'skill_group_penetration_rate_2016',"skill_group_penetration_rate_2017","skill_group_penetration_rate_2018","skill_group_rank_2015","skill_group_rank_2016","skill_group_rank_2017","skill_group_rank_2018"]]

In [16]:
#Step 2 combine industry_migration with final_skill_part
industry_combination=pd.merge(industry_migration,final_skill_part,how='left',on=['isic_section_name','industry_name'])
industry_combination=industry_combination.drop(columns=['net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018'])

In [17]:
skill_combination=pd.merge(skill_migration,final_skill_part,how='left',on=['skill_group_category','skill_group_name'])
# skill_combination=skill_combination.drop(columns=['net_per_10K_2015','net_per_10K_2016','net_per_10K_2017','net_per_10K_2018',])
skill_combination=skill_combination.drop(columns=['skill_group_penetration_rate_2015','skill_group_penetration_rate_2016','skill_group_penetration_rate_2017','skill_group_penetration_rate_2018'])
skill_combination=skill_combination.drop(columns=['skill_group_rank_2015','skill_group_rank_2016','skill_group_rank_2017','skill_group_rank_2018'])

In [18]:
#Step 3 Combine industry and skill dataset
ind_skill_combination=pd.merge(skill_combination,industry_combination,how='inner',on=['country_name','wb_income','wb_region','isic_section_name','industry_name','skill_group_category','skill_group_name'])

In [19]:
#Step 3 Combine education sheet
education=pd.merge(education_skill,education_major,how='inner',on=['Program'])


In [20]:
# Final Combine
final_dataset=pd.merge(ind_skill_combination,education,how='inner',on=['skill_group_name'])

In [23]:
final_dataset.to_csv(r'/Users/tanya/Desktop/IS590 OM/OM DATASETS/export_dataframe.csv')